In [1]:
import pandas as pd
import warnings
import shutil
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException
import selenium.webdriver.support.expected_conditions as EC
import time 
from datetime import datetime
from glob import glob
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm.notebook import tqdm as tqdm_notebook
import json
import os
import requests
import sys
from collections import ChainMap
import csv
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import traceback
from werkzeug.utils import secure_filename
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException, WebDriverException, RemoteDriverServerException
import sys
sys.path.append('../')
from config import Marco
from config import chromeSetup,get_selected_cols_df,ensure_dir,get_Drivers,encode_search_word,enter_sys_value,update_iod_tms_with_now,transform
marco = Marco()

import warnings
warnings.filterwarnings("ignore")
class FedExShipmentTracking:
    def __init__(self, df, col_name = 'Pro No', verbose = True):
        self.df = df
        self.shipment_ids = self.df[col_name].values
        self.verbose = verbose
        self.url = 'https://www.fedex.com/trackingCal/track'
        self.headers = {
                    'Connection': 'keep-alive',
                    'Pragma': 'no-cache',
                    'Cache-Control': 'no-cache',
                    'Accept': '*/*',
                    'Accept-Encoding': 'gzip, deflate, br',
                    'Accept-Language': 'en-US,en;q=0.9',
                    "Accept" : "application/json, text/javascript, */*; q=0.01",
                    'Connection': 'keep-alive',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
                    'Host': 'www.fedex.com',
                    'Origin': 'https://www.fedex.com',

                    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
                    'X-Requested-With' : "XMLHttpRequest",
                    'Cookie': 'fdx_cbid=30036807161594754925222750021081; fdx_locale=en_US; fdx_redirect=en-us; cc_path=us; s_ecid=MCMID%7C91240026263101807612795946297234918876; _abck=E6BF6C129F738D2B9D1C7477355EE277~0~YAAQL5EvFwBtVC1zAQAAAwvNTgSEKQJaNIT46+OyW/Rl1EiG2zsHB8tUiwjmoECJgd2c5DwAVVZJqwRE1WyaZUW6CBEIrzjTPlAfDtIA0eWI1PZx8cmmPMA8K043L/XtLVvra0HYHOpMsVBS/T9heRtUPhD61LP9fCtzlX8ASW2c/LueOQF0WnVI6fhEBpzT7lxVey3V7rIXOIdQx785tSS1+3g9TOPU74tgm0TvJyIpOTjLhX27RoU7wo+mzSD5HKwhzKC3pckP0hjKF1EGAXRyh7NQTD4m08aQDFxo0f0BS7BbSbLuVXHDuKdgHLRb5nJkiLs4~-1~-1~-1; _gcl_au=1.1.494437621.1594754928; tracking_locale=en_US; wdpl_id=30036807161594754925222750021081_1594754941226; optimizelyEndUserId=oeu1594842710172r0.13632731098668605; s_vi=[CS]v1|2F8820BB8515E1A5-6000073AEC279C6F[CE]; siteDC=edc; mbox=PC#1595273312475-908373.35_0#1596830577|session#1595620934343-546912#1595622837; Rbt=f0; AMCVS_1E22171B520E93BF0A490D44%40AdobeOrg=1; Nina-nina-fedex-session=%7B%22loginStatus%22%3A%22loggedOut%22%2C%22locale%22%3A%22en_us%22%2C%22lcstat%22%3Afalse%7D; s_sq=%5B%5BB%5D%5D; xacc=US; bm_sz=4537EBE6509A5D440AD47ED3F80C79BE~YAAQr3lGaOb59X9zAQAAPkn1iwjilFEKU/Hxbvqfeprv1pCh3PT2Me3bTAJQ12QgCNOtfHlxS+XK5Fa+X0LlfEm6kIgpH+Dlm9NQ0ypUeJ9BwqNwBuXI2IP6RGDvdEnsLnYg9JbJni8A/iP8YVPtGzSLZtucUOOqxj45znRIaMvvfQ7HLxOikNDq/2LQ4us=; ak_bmsc=CD81CB8828BE3D10205B58F77A78AF9F684679AF5C7D00006EAF1D5F61860971~plb5JNNd4MRJ+ZVIxorPq7NA1gFLL5TFJ/LEr5c2hx86FWjsmi7lF08BdqCL5Ee7PLBqfubVT/xzkPj+sl4riDqFyHe68mlzdT8V7t3XgyUTkNeg/eFURmr63IljjawrpW2KEswRN4XDAt1fnGzl3jEGV6PFsD+M6ECn5Cswi/rXsj0+axDVvBsXBdmloZeogYOnwKaYvTpNtxtPyYwrEzrRckCOGkkjzC/xtoyE1o6lID6AbzuMtxQvUEGAiqoQg5; AMCV_1E22171B520E93BF0A490D44%40AdobeOrg=1075005958%7CMCIDTS%7C18470%7CMCMID%7C91240026263101807612795946297234918876%7CMCAAMLH-1596385774%7C7%7CMCAAMB-1596385774%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1595788174s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.4.1; isTablet=false; isMobile=false; isWireless=false; level=test; aemserver=Prod-c0015884.prod.cloud.fedex.com; s_pers=%20s_vnum%3D1595822400191%2526vn%253D1%7C1595822400191%3B%20gpv_pageName%3Dus%252Fen%252Ffedex%252Funified%252Fsummarypage%7C1595782777780%3B%20s_nr%3D1595780977791-Repeat%7C1627316977791%3B%20s_invisit%3Dtrue%7C1595782777802%3B%20s_dfa%3Dfedexglbl%252Cfedexus%7C1595782797031%3B; s_sess=%20s_visit%3D1%3B%20SC_LINKS%3D%3B%20s_cc%3Dtrue%3B%20s_ppv%3Dus%252Fen%252Ffedex%252Funified%252Fsummarypage%252C89%252C89%252C1461%3B%20setLink%3D%3B; bm_sv=4EFB739A657166D148018747EECAE444~EEeDC8RhatPCxiJEcuvlyKw8uW9YjuNB0LsSrvXfL/4RUZTVIwcQhJSOOddouq0QvxUkB0laCZWErSpRDgmVOcGGbYaEHeDEh/JdTumXJZ3ol/DQnfjjhX6/zG1XHpRXTeqIIk+l98DMBDWQKTaMT4fzKmyVbors5C8i0jX6GsA='
                    } 

    def track(self, max_amount_track = 30):
        print(" start !!!!")
        print("we have {} fedex tracking number".format(len(self.df)))
        final_data = []
        for idx in tqdm_notebook(range(0, len(self.df),max_amount_track)):
            if idx + 30 <= len(self.df):
                data = self.encode(self.shipment_ids[idx:idx+30])
                response = self.crawl(data)
            else:
                data = self.encode(self.shipment_ids[idx:])
                response = self.crawl(data)
            
            bt_result = self.parse(response)
            final_data.extend(bt_result)
            time.sleep(1.5)
        return final_data
  
    def encode(self,ids):
        form_data = {"data":{"TrackPackagesRequest":{
                    "appType":"WTRK",
                    "appDeviceType":"DESKTOP",
                    "supportHTML":True,
                    "supportCurrentLocation":True,
                    "uniqueKey":"",
                    "processingParameters":{},
                    "trackingInfoList":[{"trackNumberInfo":{"trackingNumber":str(i),"trackingQualifier":"","trackingCarrier":""}} for i in ids]}
                            },
                    "action":"trackpackages",
                    "locale":"en_US",
                    "version":"1",
                    "format":"json",
                    }

        data = 'data='+ json.dumps(form_data['data'],separators=(',',':')) + '&action=trackpackages&locale=en_US&version=1&format=json'
        return data
        
    def crawl(self,data):
        res_id= requests.post(self.url,data=data,headers = self.headers)
        respond = res_id.json()
        return respond
    
    def parse(self, respond): ## list of dict 
        packages  = respond['TrackPackagesResponse'].get('packageList',[])
        if self.verbose:
            print(f"{len(packages)} loads tracked !!!")
        batch_result = []
        for p in packages:
            data = [p['displayTrackingNbr'],
                    p['scanEventList'][0]['status'],
                    p['displayEstDeliveryDateTime'] if p['displayActDeliveryDateTime'] == "" else p['displayActDeliveryDateTime'],
                    p['scanEventList'][0]['scanDetails']]
            if self.verbose:
                print(data)
                
            batch_result.append(
                {data[0]:{
                    "Action":data[1],
                    "Status" : data[2],
                    "Details":data[3]
                }}
            )
            
        return batch_result
    

class SeleniumException(Exception):
    screenshot_path = ''
    error = ''
    exc_module = ''

    def __init__(self, error, screenshot_path=None):
        self.screenshot_path = screenshot_path
        self.error = error


def logger(f):
    def inner(self, *args, **kwargs):
        try:
            return f(self, *args, **kwargs)
        except Exception:
            file_path = os.getenv('SCREENSHOTS_FOLDER') + secure_filename(str(datetime.datetime.now()) + '.png')
            self.driver.save_screenshot(file_path)
            raise SeleniumException(traceback.format_exc(), file_path)
    return inner



class RobotWebdriver:
    def __init__(self):
        selenium_path = ChromeDriverManager().install()
        chrome_options = Options()
        # chrome_options.add_argument("--no-sandbox")
        # chrome_options.add_argument("--disable-dev-shm-usage")
        
        if os.getenv('HEADLESS') == '1':
            chrome_options.add_argument("--headless")

        # chrome_options.add_argument("--no-sandbox")
        # chrome_options.add_argument("--disable-dev-shm-usage")
        self.driver = webdriver.Chrome(executable_path=selenium_path, options=chrome_options)
        # self.driver.set_window_size(870, 2300)
        # Save useful imports onto driver
        self.driver.EC = EC
        self.driver.By = By
        self.driver.WebDriverWait = WebDriverWait
        self.driver.TimeoutException = TimeoutException
        self.driver.UnexpectedAlertPresentException = UnexpectedAlertPresentException
        self.driver.WebDriverException = WebDriverException
        self.driver.RemoteDriverServerException = RemoteDriverServerException

    @logger
    def pay(self, invoice):
        plugid = invoice["robotina_plugin"]
        if plugid not in plugins.keys():
            print(f"Plugin {plugid} is not loaded")
            return None
        
        plug = plugins[plugid]
        res = plug.pay(self.driver, invoice)
        self.pay_count += 1
        return res
    
    def close_driver(self):
        self.driver.close()

   
def get_latest_df(path, file_prefix):
    file_path = []
    try: 
        file_path = sorted(glob(os.path.join(path, file_prefix + "_*.csv")))
        return pd.read_csv(file_path[-1])
    except Exception as e:
        pass

          
        
def getCodes(carrier_code):
    dit = {
        "RDWY":[
            'RDWY_CRA','RDWY_FON','RDWY_FTW','RDWY_POR','RDWY_ROM',
               ],
        "UTP2":["UTP2"],

    }    
    
    return dit[carrier_code]

def clean(text):
    return "".join([ i for i in text if i.isdigit() or i == "X"])



             
def check(carrier_code_list, path = "../daily_route_data/",pre_fix = "IOD_MONITORING_DETAIL_US"):  
    _df = get_latest_df(os.getenv("DATA_FILE_PATH",path),pre_fix)
    
    for carrier_code in carrier_code_list:
        codes = getCodes(carrier_code)
        df = _df[_df['Carrier Code'].isin(codes)]
        result = []
        loads = df["Pro No"].to_list()
        
        if not hasattr(sys,'robotina_webdriver'): sys.robotina_webdriver = RobotWebdriver()
        wd = sys.robotina_webdriver
        
        if carrier_code == "RDWY":
            for load in loads:
                try:
                    url = f'https://my.yrc.com/tools/track/shipments?referenceNumber={load}&referenceNumberType=PRO&time={int(time.mktime(datetime.now().timetuple()))}'
                    wd.driver.get(url)
                    time.sleep(2)
                    t = wd.driver.WebDriverWait(wd.driver, 3).until(wd.driver.EC.visibility_of_element_located((wd.driver.By.CLASS_NAME, "panel-body")))
                    time.sleep(0.3)
                    t = wd.driver.find_elements_by_class_name("panel-body")
#                     time.sleep(2)
#                     t = wd.driver.find_elements_by_class_name("panel-body")
                    if len(t) > 1:
                        d = t[1].text.split("\n")
                        data = {}
                        for k in range(0,len(d),2):
                            if k == len(d): continue
                            data[d[k]] = d[k+1]
                        data['YRC Freight PRO #:'] = clean(data['YRC Freight PRO #:'] )
                        result.append({
                            data['YRC Freight PRO #:']:data 
                        })
                        
                except:
                    print(load, "   not found ! ")
            result = dict(ChainMap(*result))
            print(f"Carrier result checked :  ",len(result))

            timestamp = str(datetime.now())        
            
            df = df[marco.colsSend2Carrier]

            today_date = timestamp[:10]
            timestamp = "".join([d for d  in timestamp if d.isdigit()])
            ensure_dir(today_date)
            df['Status'] = df['Pro No'].apply(lambda x : result[x]['Status:'] if result.get(x) and result.get(x).get("Status:")  else "null")
            df['Status Updated:'] = df['Pro No'].apply(lambda x : result[x]['Status Updated:'] if result.get(x) and result.get(x).get('Status Updated:') else "null")
            df['Delivered Date'] = df['Pro No'].apply(lambda x : result[x]['Delivered Date'] if result.get(x) and result.get(x).get('Delivered Date') else "null" )
            df['Est. Delivery Date:'] = df['Pro No'].apply(lambda x : result[x]['Est. Delivery Date:'] if result.get(x) and result.get(x).get('Est. Delivery Date:') else "null" )
            df['Appointment Date:'] = df['Pro No'].apply(lambda x : result[x]['Appointment Date:'] if result.get(x) and result.get(x).get('Appointment Date:') else "null" )

            df.to_excel(os.path.join("table_result",today_date,f"report_RDWY_{timestamp[8:12]}.xlsx"),index=False,engine='xlsxwriter')
                    
        if carrier_code == "UTP2":
            wd.driver.get("https://client.freightapp.com/tms/template/orderTracking.html")
            time.sleep(0.3)
            dit = {}
            for load in loads:
                
                search = wd.driver.find_element_by_xpath('//*[@id="tracking_no"]')
                search.clear()
                time.sleep(0.1)
                search.send_keys(load)
                search.send_keys(Keys.ENTER)
                time.sleep(0.2)
                try:
                    log_list = wd.driver.find_element_by_xpath('//*[@id="log_list"]')
                    state = log_list.text.split("\n")
                    details = wd.driver.find_element_by_xpath('//*[@id="details_section"]/div/div').text.split("\n")
                    idx = details[1].index('Delivery Appointment')
                    dit[load] = {
                        "update date:": state[1],
                        "status" : state[2],
                        "appt":details[1][idx:]
                    }
                    
                except:
                    print(load, "   not found ! ")
            timestamp = str(datetime.now())        
            today_date = timestamp[:10]
            timestamp = "".join([d for d  in timestamp if d.isdigit()])
            ensure_dir(os.path.join("table_result",today_date))
            df = df[marco.colsSend2Carrier]
            df['update date'] = df['Pro No'].apply(lambda x : dit[x]['update date:'] if dit.get(x) else "NA")
            df['status'] = df['Pro No'].apply(lambda x : " ".join(dit[x]['status'].split()[2:]) if dit.get(x) else "NA")
            df['appt time'] = df['Pro No'].apply(lambda x : dit[x]['appt'] if dit.get(x) else "NA")
            df['delivered_completed'] = df['status'].apply(lambda x : "Yes" if "Delivery Completed" in x else "No")
            df['Stop Seq'] = df['From Address'].apply(lambda x : x[-12:][:2])
            df.rename(columns= {
                'Stop Seq':"Ship State"
            },inplace = True)

            df = df.sort_values("Ship State")
            df.to_excel(os.path.join("table_result",today_date,f"report_UTP2_{timestamp[8:12]}.xlsx"),index=False,engine='xlsxwriter')
        
    wd.close_driver()



In [5]:
check(['RDWY'])



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\marco.wang\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


Carrier result checked :   115


In [6]:
options = chromeSetup()
driver = get_Drivers(options)
original_window = driver.current_window_handle
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break

get browsers!!
log in !!
MU
Parcel
TL
Total
Total
Done!!


In [ ]:
wait = WebDriverWait(driver, 300, poll_frequency=10, ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException])
element = wait.until(EC.element_to_be_clickable((By.ID, "iodMonitoringDetailSaveExcel")))
        
    
time.sleep(60)
# download = driver.find_element_by_id("iodMonitoringDetailSaveExcel")
element.click()

In [ ]:
### download file to destination
downloaded_file = sorted(glob("../../../Downloads/IOD_MONITORING_DETAIL_US_*.csv"))[-1]
shutil.copy2(downloaded_file, "./data")
shutil.copy2(downloaded_file, "../dailyWork/daily_route_data")

In [ ]:
browers = driver
browers.maximize_window()

searchLoads = browers.find_element_by_id("searchLoadId")
searchLoads.clear()
time.sleep(0.1)
searchLoads.send_keys(encode_search_word(todays_left))
time.sleep(0.5)

searchButton = browers.find_element_by_id("searchButton")
searchButton.click()
time.sleep(60)

In [ ]:
update_iod_tms_with_now(browers)